## This script reads CESM2 Soil Moisture data downloaded from Casper in NCAR (Fig S6)

## Import Libraries

In [1]:
import numpy as np
import pandas as pd
import datetime
import xarray as xr
import seaborn as sns
import warnings
from matplotlib import pyplot as plt 
import matplotlib as mpl
import shapefile as shp
import cartopy.io.shapereader as shpreader
from numpy import linspace
from numpy import meshgrid
import matplotlib.colors as colors
from numpy import array, ma
import cartopy.crs as ccrs
import matplotlib.cm as cm
import cartopy.feature as cfeature

## Read Lat, Lon data for the selected Domain processed from "GLADE" in Casper

## Lat

In [2]:
file_LatUs = 'C:/rezaul/1.academic/1.research/2.project/mrb/dt/cesm/cesm2LE/param/precip/mon/latsUsArr.npy'
lats_US = np.load(file_LatUs)
print(lats_US.shape)
print(lats_US)

latUs_cesm2 = lats_US

(32,)
[24.973822 25.916231 26.858639 27.801046 28.743456 29.685863 30.628273
 31.57068  32.51309  33.455498 34.397907 35.340313 36.282722 37.225132
 38.167538 39.109947 40.052357 40.994766 41.937172 42.87958  43.82199
 44.764397 45.706806 46.649216 47.59162  48.53403  49.47644  50.41885
 51.361256 52.303665 53.246075 54.18848 ]


## Lon

In [3]:
file_LonUs = 'C:/rezaul/1.academic/1.research/2.project/mrb/dt/cesm/cesm2LE/param/precip/mon/lonsUsArr.npy'
lons_US = np.load(file_LonUs)
print(lons_US.shape)
print(lons_US)

lonUs_cesm2 = lons_US

(56,)
[230.   231.25 232.5  233.75 235.   236.25 237.5  238.75 240.   241.25
 242.5  243.75 245.   246.25 247.5  248.75 250.   251.25 252.5  253.75
 255.   256.25 257.5  258.75 260.   261.25 262.5  263.75 265.   266.25
 267.5  268.75 270.   271.25 272.5  273.75 275.   276.25 277.5  278.75
 280.   281.25 282.5  283.75 285.   286.25 287.5  288.75 290.   291.25
 292.5  293.75 295.   296.25 297.5  298.75]


### Historical (1930-2014): 1020 months

In [4]:
file_histSel_ensMean = 'C:/rezaul/1.academic/1.research/2.project/mrb/dt/cesm/cesm2LE/param/soilMoist/mon/monSoilMoistUsHistSelEnsMean.npy'
file_histSel_ensMean_loaded = np.load(file_histSel_ensMean)
print(file_histSel_ensMean_loaded.shape)

(1020, 32, 56)


### Scenario (2105-2100): 1032 months

In [5]:
file_scen_ensMean = 'C:/rezaul/1.academic/1.research/2.project/mrb/dt/cesm/cesm2LE/param/soilMoist/mon/monSoilMoistUsScenEnsMean.npy'
file_scen_ensMean_loaded = np.load(file_scen_ensMean)
print(file_scen_ensMean_loaded.shape)

(1032, 32, 56)


## Concatenate two data arrays

In [6]:
param_histSel_scen_ensMean_Us = np.concatenate((file_histSel_ensMean_loaded, file_scen_ensMean_loaded), axis=0)
print(param_histSel_scen_ensMean_Us.shape)

(2052, 32, 56)


## Writing in an Xarray Data array

In [7]:
print(param_histSel_scen_ensMean_Us.shape)

(2052, 32, 56)


In [8]:
t_histSel_scen, y_Us, x_Us = param_histSel_scen_ensMean_Us.shape

print(t_histSel_scen)
print(y_Us)
print(x_Us)

2052
32
56


In [9]:
Month_steps = [x for x in range(t_histSel_scen)]
#print(Month_steps)


In [10]:
param_histSel_scen_ensMean_Us_xarray = xr.DataArray(param_histSel_scen_ensMean_Us, coords=[Month_steps, latUs_cesm2, lonUs_cesm2], dims=["Month_steps","Lat","Lon"])
print(param_histSel_scen_ensMean_Us_xarray.shape)
#print(param_histSel_scen_ensMean_Us_xarray)

(2052, 32, 56)


## Masking files for the US Domain only

## Mississippi

In [11]:
MaskFile_Missi_to_load = 'C:/rezaul/1.academic/1.research/2.project/mrb/dt/shapeFile/mask/mrb_us/mask_missiWri_us_CESM_longThree.nc'
print(MaskFile_Missi_to_load)

C:/rezaul/1.academic/1.research/2.project/mrb/dt/shapeFile/mask/mrb_us/mask_missiWri_us_CESM_longThree.nc


In [12]:
MaskFile_Missi = xr.open_dataset(MaskFile_Missi_to_load)['Mask']
print(MaskFile_Missi)

<xarray.DataArray 'Mask' (Lat: 32, Lon: 56)>
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])
Coordinates:
  * Lat      (Lat) float32 24.97 25.92 26.86 27.8 ... 51.36 52.3 53.25 54.19
  * Lon      (Lon) float32 230.0 231.2 232.5 233.8 ... 295.0 296.2 297.5 298.8


C:\Users\muham\anaconda3\lib\site-packages\xarray\backends\plugins.py:61: RuntimeWarning: Engine 'rasterio' loading failed:
DLL load failed while importing _base: The specified procedure could not be found.
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


In [13]:
param_masked_Missi = param_histSel_scen_ensMean_Us_xarray.where(MaskFile_Missi>0)
print(param_masked_Missi.shape)
#print(param_masked_Missi)

(2052, 32, 56)


## Missouri

In [14]:
# Make commented of the following lines if you want to make the Mask file for the first time
# C:/rezaul/1.academic/1.research/2.project/mrb/dt/shapeFile/mask/mrb_us/mask_missiWri_us_CESM_longThree.nc
MaskFile_Missouri_to_load = 'C:/rezaul/1.academic/1.research/2.project/mrb/dt/shapeFile/mask/mrb_us/mask_missouri_us_CESM_longThree.nc'
print(MaskFile_Missouri_to_load)

C:/rezaul/1.academic/1.research/2.project/mrb/dt/shapeFile/mask/mrb_us/mask_missouri_us_CESM_longThree.nc


In [15]:
MaskFile_Missouri = xr.open_dataset(MaskFile_Missouri_to_load)['Mask']
print(MaskFile_Missouri)

<xarray.DataArray 'Mask' (Lat: 32, Lon: 56)>
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])
Coordinates:
  * Lat      (Lat) float32 24.97 25.92 26.86 27.8 ... 51.36 52.3 53.25 54.19
  * Lon      (Lon) float32 230.0 231.2 232.5 233.8 ... 295.0 296.2 297.5 298.8


In [16]:
param_masked_Missouri = param_histSel_scen_ensMean_Us_xarray.where(MaskFile_Missouri>0)
print(param_masked_Missouri.shape)
#print(param_masked_Missouri)

(2052, 32, 56)


## Arkansas

In [17]:
MaskFile_Arkansas_to_load = 'C:/rezaul/1.academic/1.research/2.project/mrb/dt/shapeFile/mask/mrb_us/mask_arkansas_us_CESM_longThree.nc'
print(MaskFile_Arkansas_to_load)

C:/rezaul/1.academic/1.research/2.project/mrb/dt/shapeFile/mask/mrb_us/mask_arkansas_us_CESM_longThree.nc


In [18]:
MaskFile_Arkansas = xr.open_dataset(MaskFile_Arkansas_to_load)['Mask']
print(MaskFile_Arkansas)

<xarray.DataArray 'Mask' (Lat: 32, Lon: 56)>
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])
Coordinates:
  * Lat      (Lat) float32 24.97 25.92 26.86 27.8 ... 51.36 52.3 53.25 54.19
  * Lon      (Lon) float32 230.0 231.2 232.5 233.8 ... 295.0 296.2 297.5 298.8


In [19]:
param_masked_Arkansas = param_histSel_scen_ensMean_Us_xarray.where(MaskFile_Arkansas>0)
print(param_masked_Arkansas.shape)
#print(param_masked_Arkansas)

(2052, 32, 56)


## Ohio

In [20]:
MaskFile_Ohio_to_load = 'C:/rezaul/1.academic/1.research/2.project/mrb/dt/shapeFile/mask/mrb_us/mask_ohio_us_CESM_longThree.nc'
print(MaskFile_Ohio_to_load)

C:/rezaul/1.academic/1.research/2.project/mrb/dt/shapeFile/mask/mrb_us/mask_ohio_us_CESM_longThree.nc


In [21]:
MaskFile_Ohio = xr.open_dataset(MaskFile_Ohio_to_load)['Mask']
print(MaskFile_Ohio)

<xarray.DataArray 'Mask' (Lat: 32, Lon: 56)>
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])
Coordinates:
  * Lat      (Lat) float32 24.97 25.92 26.86 27.8 ... 51.36 52.3 53.25 54.19
  * Lon      (Lon) float32 230.0 231.2 232.5 233.8 ... 295.0 296.2 297.5 298.8


In [22]:
param_masked_Ohio = param_histSel_scen_ensMean_Us_xarray.where(MaskFile_Ohio>0)
print(param_masked_Ohio.shape)
#print(param_masked_Ohio)

(2052, 32, 56)


## Masking for Lower Mississippi Basin

In [23]:
param_masked_lowMissi = param_histSel_scen_ensMean_Us_xarray.sel(Lat=slice(30.,37.), Lon = slice(268., 271.))
print(param_masked_lowMissi.shape)

(2052, 7, 2)


## The masked rasters for different basins

In [24]:
print(param_masked_Missi.shape)
print(param_masked_Missouri.shape)
print(param_masked_Arkansas.shape)
print(param_masked_Ohio.shape)
print('')
print(param_masked_lowMissi.shape)

(2052, 32, 56)
(2052, 32, 56)
(2052, 32, 56)
(2052, 32, 56)

(2052, 7, 2)


## Time series for basin-average Param value: Monthly resolution

In [25]:
param_masked_ensMean_basinAvg_Missi = np.nanmean(param_masked_Missi, axis=(1,2))
print(param_masked_ensMean_basinAvg_Missi.shape)

(2052,)


In [26]:
param_masked_ensMean_basinAvg_Missouri = np.nanmean(param_masked_Missouri, axis=(1,2))
print(param_masked_ensMean_basinAvg_Missouri.shape)

(2052,)


In [27]:
param_masked_ensMean_basinAvg_Arkansas = np.nanmean(param_masked_Arkansas, axis=(1,2))
print(param_masked_ensMean_basinAvg_Arkansas.shape)

(2052,)


In [28]:
param_masked_ensMean_basinAvg_Ohio = np.nanmean(param_masked_Ohio, axis=(1,2))
print(param_masked_ensMean_basinAvg_Ohio.shape)

(2052,)


In [29]:
param_masked_ensMean_basinAvg_lowMissi = np.nanmean(param_masked_lowMissi, axis=(1,2))
print(param_masked_ensMean_basinAvg_lowMissi.shape)

(2052,)


In [30]:
t_histSel_scen = param_masked_ensMean_basinAvg_Missi.shape[0]
print(t_histSel_scen)

2052


In [31]:
Month_steps = [x for x in range(t_histSel_scen)]

## For Seasonal Values

In [32]:
def Season(value):
    if value <= 2:
        return "Winter"
    if 3 <= value <= 5:
        return "Spring"
    elif 6 <= value <= 8:
        return "Summer"
    elif 9 <= value <= 11:
        return "Fall"
    elif value == 12:
        return "Winter"

In [33]:
dates_histSelScen = xr.cftime_range(start='193001',freq='1M', periods=2052, calendar='noleap').strftime('%Y-%m').to_list()
print(dates_histSelScen[:3], dates_histSelScen[-1])


['1930-01', '1930-02', '1930-03'] 2100-12


# Missouri basin

In [34]:
print(param_masked_ensMean_basinAvg_Missouri.shape)

(2052,)


In [35]:

param_masked_ensMean_basinAvg_Missouri_xarray = xr.DataArray(param_masked_ensMean_basinAvg_Missouri, coords=[Month_steps], dims=["Month_steps"])
print(param_masked_ensMean_basinAvg_Missouri_xarray.shape)
print(param_masked_ensMean_basinAvg_Missouri_xarray)

#df['Day_steps'] = dates_histSelScen
param_masked_ensMean_basinAvg_Missouri_xarray['Month_steps'] = pd.to_datetime(dates_histSelScen)
print(param_masked_ensMean_basinAvg_Missouri_xarray.shape)
print(param_masked_ensMean_basinAvg_Missouri_xarray.head(3))

#df = disch_histSel_Scen_ensMean_vicks_xarray.to_dataframe(name='Discharge').unstack().reset_index()
df_param_Missouri = param_masked_ensMean_basinAvg_Missouri_xarray.to_dataframe(name='Soil_moisture').reset_index()
print(df_param_Missouri.shape)
print(df_param_Missouri.head(3))

df_param_Missouri['Season_year'] = df_param_Missouri.Month_steps.dt.year.where(df_param_Missouri.Month_steps.dt.month < 12, df_param_Missouri.Month_steps.dt.year + 1)
print(df_param_Missouri.shape)
print(df_param_Missouri.head(15))


df_param_Missouri['Season_mon'] = df_param_Missouri.Month_steps.dt.month.where(df_param_Missouri.Month_steps.dt.month == 1, df_param_Missouri.Month_steps.dt.month)
print(df_param_Missouri.shape)
print(df_param_Missouri.head(15))


df_param_Missouri['Season'] = df_param_Missouri['Season_mon'].map(Season)
#display(df_param_Missouri.head())
print(df_param_Missouri.shape)
print(df_param_Missouri.head(15))

df_param_winter_Missouri_withOneExtra = df_param_Missouri[df_param_Missouri["Season"] == 'Winter'].groupby('Season_year').mean('Season_year')
print(df_param_winter_Missouri_withOneExtra.shape)
print(df_param_winter_Missouri_withOneExtra)

## to Delete the last (unwanted) row
df_param_winter_Missouri = df_param_winter_Missouri_withOneExtra.iloc[:-1 , :]
print(df_param_winter_Missouri.shape)
print(df_param_winter_Missouri.tail())

df_param_spring_Missouri = df_param_Missouri[df_param_Missouri["Season"] == 'Spring'].groupby('Season_year').mean('Season_year')
print(df_param_spring_Missouri.shape)
print(df_param_spring_Missouri)

df_param_summer_Missouri = df_param_Missouri[df_param_Missouri["Season"] == 'Summer'].groupby('Season_year').mean('Season_year')
print(df_param_summer_Missouri.shape)
print(df_param_summer_Missouri)

df_param_fall_Missouri = df_param_Missouri[df_param_Missouri["Season"] == 'Fall'].groupby('Season_year').mean('Season_year')
print(df_param_fall_Missouri.shape)
print(df_param_fall_Missouri)

param_winter_Missouri = df_param_winter_Missouri['Soil_moisture']
print(param_winter_Missouri.shape)
print(param_winter_Missouri)

param_spring_Missouri = df_param_spring_Missouri['Soil_moisture']
print(param_spring_Missouri.shape)
print(param_spring_Missouri)

param_summer_Missouri = df_param_summer_Missouri['Soil_moisture']
print(param_summer_Missouri.shape)
print(param_summer_Missouri)

param_fall_Missouri = df_param_fall_Missouri['Soil_moisture']
print(param_fall_Missouri.shape)
print(param_fall_Missouri)



(2052,)
<xarray.DataArray (Month_steps: 2052)>
array([33.65536699, 34.50348006, 35.4149634 , ..., 21.96295739,
       25.49797002, 28.70184618])
Coordinates:
  * Month_steps  (Month_steps) int32 0 1 2 3 4 5 ... 2047 2048 2049 2050 2051
(2052,)
<xarray.DataArray (Month_steps: 3)>
array([33.65536699, 34.50348006, 35.4149634 ])
Coordinates:
  * Month_steps  (Month_steps) datetime64[ns] 1930-01-01 1930-02-01 1930-03-01
(2052, 2)
  Month_steps  Soil_moisture
0  1930-01-01      33.655367
1  1930-02-01      34.503480
2  1930-03-01      35.414963
(2052, 3)
   Month_steps  Soil_moisture  Season_year
0   1930-01-01      33.655367         1930
1   1930-02-01      34.503480         1930
2   1930-03-01      35.414963         1930
3   1930-04-01      33.913174         1930
4   1930-05-01      30.515319         1930
5   1930-06-01      27.837017         1930
6   1930-07-01      23.522575         1930
7   1930-08-01      21.015738         1930
8   1930-09-01      22.211177         1930
9   1930-10-01 

## Arkansas basin

In [36]:
print(param_masked_ensMean_basinAvg_Arkansas.shape)

(2052,)


In [37]:
param_masked_ensMean_basinAvg_Arkansas_xarray = xr.DataArray(param_masked_ensMean_basinAvg_Arkansas, coords=[Month_steps], dims=["Month_steps"])
print(param_masked_ensMean_basinAvg_Arkansas_xarray.shape)
print(param_masked_ensMean_basinAvg_Arkansas_xarray)

#df['Day_steps'] = dates_histSelScen
param_masked_ensMean_basinAvg_Arkansas_xarray['Month_steps'] = pd.to_datetime(dates_histSelScen)
print(param_masked_ensMean_basinAvg_Arkansas_xarray.shape)
print(param_masked_ensMean_basinAvg_Arkansas_xarray.head(3))

#df = disch_histSel_Scen_ensMean_vicks_xarray.to_dataframe(name='Discharge').unstack().reset_index()
df_param_Arkansas = param_masked_ensMean_basinAvg_Arkansas_xarray.to_dataframe(name='Soil_moisture').reset_index()
print(df_param_Arkansas.shape)
print(df_param_Arkansas.head(3))

df_param_Arkansas['Season_year'] = df_param_Arkansas.Month_steps.dt.year.where(df_param_Arkansas.Month_steps.dt.month < 12, df_param_Arkansas.Month_steps.dt.year + 1)
print(df_param_Arkansas.shape)
print(df_param_Arkansas.head(15))


df_param_Arkansas['Season_mon'] = df_param_Arkansas.Month_steps.dt.month.where(df_param_Arkansas.Month_steps.dt.month == 1, df_param_Arkansas.Month_steps.dt.month)
print(df_param_Arkansas.shape)
print(df_param_Arkansas.head(15))

df_param_Arkansas['Season'] = df_param_Arkansas['Season_mon'].map(Season)
#display(df_param_Arkansas.head())
print(df_param_Arkansas.shape)
print(df_param_Arkansas.head(15))

df_param_winter_Arkansas_withOneExtra = df_param_Arkansas[df_param_Arkansas["Season"] == 'Winter'].groupby('Season_year').mean('Season_year')
print(df_param_winter_Arkansas_withOneExtra.shape)
print(df_param_winter_Arkansas_withOneExtra)

## to Delete the last (unwanted) row
df_param_winter_Arkansas = df_param_winter_Arkansas_withOneExtra.iloc[:-1 , :]
print(df_param_winter_Arkansas.shape)
print(df_param_winter_Arkansas.tail())
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

## Split into Seasons
df_param_spring_Arkansas = df_param_Arkansas[df_param_Arkansas["Season"] == 'Spring'].groupby('Season_year').mean('Season_year')
print(df_param_spring_Arkansas.shape)
print(df_param_spring_Arkansas)

df_param_summer_Arkansas = df_param_Arkansas[df_param_Arkansas["Season"] == 'Summer'].groupby('Season_year').mean('Season_year')
print(df_param_summer_Arkansas.shape)
print(df_param_summer_Arkansas)

df_param_fall_Arkansas = df_param_Arkansas[df_param_Arkansas["Season"] == 'Fall'].groupby('Season_year').mean('Season_year')
print(df_param_fall_Arkansas.shape)
print(df_param_fall_Arkansas)

param_winter_Arkansas = df_param_winter_Arkansas['Soil_moisture']
print(param_winter_Arkansas.shape)
print(param_winter_Arkansas)

param_spring_Arkansas = df_param_spring_Arkansas['Soil_moisture']
print(param_spring_Arkansas.shape)
print(param_spring_Arkansas)

param_summer_Arkansas = df_param_summer_Arkansas['Soil_moisture']
print(param_summer_Arkansas.shape)
print(param_summer_Arkansas)

param_fall_Arkansas = df_param_fall_Arkansas['Soil_moisture']
print(param_fall_Arkansas.shape)
print(param_fall_Arkansas)


(2052,)
<xarray.DataArray (Month_steps: 2052)>
array([31.3086846 , 31.61837143, 30.40373449, ..., 21.38649245,
       24.13050921, 26.81239025])
Coordinates:
  * Month_steps  (Month_steps) int32 0 1 2 3 4 5 ... 2047 2048 2049 2050 2051
(2052,)
<xarray.DataArray (Month_steps: 3)>
array([31.3086846 , 31.61837143, 30.40373449])
Coordinates:
  * Month_steps  (Month_steps) datetime64[ns] 1930-01-01 1930-02-01 1930-03-01
(2052, 2)
  Month_steps  Soil_moisture
0  1930-01-01      31.308685
1  1930-02-01      31.618371
2  1930-03-01      30.403734
(2052, 3)
   Month_steps  Soil_moisture  Season_year
0   1930-01-01      31.308685         1930
1   1930-02-01      31.618371         1930
2   1930-03-01      30.403734         1930
3   1930-04-01      29.437229         1930
4   1930-05-01      27.174727         1930
5   1930-06-01      24.609236         1930
6   1930-07-01      19.916124         1930
7   1930-08-01      18.385260         1930
8   1930-09-01      21.206165         1930
9   1930-10-01 

# Ohio basin

In [38]:
print(param_masked_ensMean_basinAvg_Ohio.shape)

(2052,)


In [39]:

param_masked_ensMean_basinAvg_Ohio_xarray = xr.DataArray(param_masked_ensMean_basinAvg_Ohio, coords=[Month_steps], dims=["Month_steps"])
print(param_masked_ensMean_basinAvg_Ohio_xarray.shape)
print(param_masked_ensMean_basinAvg_Ohio_xarray)

#df['Day_steps'] = dates_histSelScen
param_masked_ensMean_basinAvg_Ohio_xarray['Month_steps'] = pd.to_datetime(dates_histSelScen)
print(param_masked_ensMean_basinAvg_Ohio_xarray.shape)
print(param_masked_ensMean_basinAvg_Ohio_xarray.head(3))

#df = disch_histSel_Scen_ensMean_vicks_xarray.to_dataframe(name='Discharge').unstack().reset_index()
df_param_Ohio = param_masked_ensMean_basinAvg_Ohio_xarray.to_dataframe(name='Soil_moisture').reset_index()
print(df_param_Ohio.shape)
print(df_param_Ohio.head(3))

df_param_Ohio['Season_year'] = df_param_Ohio.Month_steps.dt.year.where(df_param_Ohio.Month_steps.dt.month < 12, df_param_Ohio.Month_steps.dt.year + 1)
print(df_param_Ohio.shape)
print(df_param_Ohio.head(15))
#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


df_param_Ohio['Season_mon'] = df_param_Ohio.Month_steps.dt.month.where(df_param_Ohio.Month_steps.dt.month == 1, df_param_Ohio.Month_steps.dt.month)
print(df_param_Ohio.shape)
print(df_param_Ohio.head(15))

df_param_Ohio['Season'] = df_param_Ohio['Season_mon'].map(Season)
#display(df_param_Ohio.head())
print(df_param_Ohio.shape)
print(df_param_Ohio.head(15))

df_param_winter_Ohio_withOneExtra = df_param_Ohio[df_param_Ohio["Season"] == 'Winter'].groupby('Season_year').mean('Season_year')
print(df_param_winter_Ohio_withOneExtra.shape)
print(df_param_winter_Ohio_withOneExtra)

## to Delete the last (unwanted) row
df_param_winter_Ohio = df_param_winter_Ohio_withOneExtra.iloc[:-1 , :]
print(df_param_winter_Ohio.shape)
print(df_param_winter_Ohio.tail())
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


## Split into Seasons
df_param_spring_Ohio = df_param_Missouri[df_param_Ohio["Season"] == 'Spring'].groupby('Season_year').mean('Season_year')
print(df_param_spring_Ohio.shape)
print(df_param_spring_Ohio)

df_param_summer_Ohio = df_param_Missouri[df_param_Ohio["Season"] == 'Summer'].groupby('Season_year').mean('Season_year')
print(df_param_summer_Ohio.shape)
print(df_param_summer_Ohio)

df_param_fall_Ohio = df_param_Ohio[df_param_Ohio["Season"] == 'Fall'].groupby('Season_year').mean('Season_year')
print(df_param_fall_Ohio.shape)
print(df_param_fall_Ohio)

param_winter_Ohio = df_param_winter_Ohio['Soil_moisture']
print(param_winter_Ohio.shape)
print(param_winter_Ohio)

param_spring_Ohio = df_param_spring_Ohio['Soil_moisture']
print(param_spring_Ohio.shape)
print(param_spring_Ohio)

param_summer_Ohio = df_param_summer_Ohio['Soil_moisture']
print(param_summer_Ohio.shape)
print(param_summer_Ohio)

param_fall_Ohio = df_param_fall_Ohio['Soil_moisture']
print(param_fall_Ohio.shape)
print(param_fall_Ohio)
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


(2052,)
<xarray.DataArray (Month_steps: 2052)>
array([42.62812814, 43.09809583, 41.39680373, ..., 28.26483037,
       32.98015171, 37.22468178])
Coordinates:
  * Month_steps  (Month_steps) int32 0 1 2 3 4 5 ... 2047 2048 2049 2050 2051
(2052,)
<xarray.DataArray (Month_steps: 3)>
array([42.62812814, 43.09809583, 41.39680373])
Coordinates:
  * Month_steps  (Month_steps) datetime64[ns] 1930-01-01 1930-02-01 1930-03-01
(2052, 2)
  Month_steps  Soil_moisture
0  1930-01-01      42.628128
1  1930-02-01      43.098096
2  1930-03-01      41.396804
(2052, 3)
   Month_steps  Soil_moisture  Season_year
0   1930-01-01      42.628128         1930
1   1930-02-01      43.098096         1930
2   1930-03-01      41.396804         1930
3   1930-04-01      38.958204         1930
4   1930-05-01      36.149754         1930
5   1930-06-01      33.279759         1930
6   1930-07-01      29.817754         1930
7   1930-08-01      27.336747         1930
8   1930-09-01      27.148942         1930
9   1930-10-01 

## Lower Mississippi basin

In [40]:
print(param_masked_ensMean_basinAvg_lowMissi.shape)

(2052,)


In [41]:
param_masked_ensMean_basinAvg_lowMissi_xarray = xr.DataArray(param_masked_ensMean_basinAvg_lowMissi, coords=[Month_steps], dims=["Month_steps"])
print(param_masked_ensMean_basinAvg_lowMissi_xarray.shape)
print(param_masked_ensMean_basinAvg_lowMissi_xarray)

#df['Day_steps'] = dates_histSelScen
param_masked_ensMean_basinAvg_lowMissi_xarray['Month_steps'] = pd.to_datetime(dates_histSelScen)
print(param_masked_ensMean_basinAvg_lowMissi_xarray.shape)
print(param_masked_ensMean_basinAvg_lowMissi_xarray.head(3))

#df = disch_histSel_Scen_ensMean_vicks_xarray.to_dataframe(name='Discharge').unstack().reset_index()
df_param_lowMissi = param_masked_ensMean_basinAvg_lowMissi_xarray.to_dataframe(name='Soil_moisture').reset_index()
print(df_param_lowMissi.shape)
print(df_param_lowMissi.head(3))

df_param_lowMissi['Season_year'] = df_param_lowMissi.Month_steps.dt.year.where(df_param_lowMissi.Month_steps.dt.month < 12, df_param_lowMissi.Month_steps.dt.year + 1)
print(df_param_lowMissi.shape)
print(df_param_lowMissi.head(15))
#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

df_param_lowMissi['Season_mon'] = df_param_lowMissi.Month_steps.dt.month.where(df_param_lowMissi.Month_steps.dt.month == 1, df_param_lowMissi.Month_steps.dt.month)
print(df_param_lowMissi.shape)
print(df_param_lowMissi.head(15))

df_param_lowMissi['Season'] = df_param_lowMissi['Season_mon'].map(Season)
#display(df_param_lowMissi.head())
print(df_param_lowMissi.shape)
print(df_param_lowMissi.head(15))

df_param_winter_lowMissi_withOneExtra = df_param_lowMissi[df_param_lowMissi["Season"] == 'Winter'].groupby('Season_year').mean('Season_year')
print(df_param_winter_lowMissi_withOneExtra.shape)
print(df_param_winter_lowMissi_withOneExtra)

## to Delete the last (unwanted) row
df_param_winter_lowMissi = df_param_winter_lowMissi_withOneExtra.iloc[:-1 , :]
print(df_param_winter_lowMissi.shape)
print(df_param_winter_lowMissi.tail())
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


## Split into Seasons
df_param_spring_lowMissi = df_param_lowMissi[df_param_lowMissi["Season"] == 'Spring'].groupby('Season_year').mean('Season_year')
print(df_param_spring_lowMissi.shape)
print(df_param_spring_lowMissi)

df_param_summer_lowMissi = df_param_lowMissi[df_param_lowMissi["Season"] == 'Summer'].groupby('Season_year').mean('Season_year')
print(df_param_summer_lowMissi.shape)
print(df_param_summer_lowMissi)

df_param_fall_lowMissi = df_param_lowMissi[df_param_lowMissi["Season"] == 'Fall'].groupby('Season_year').mean('Season_year')
print(df_param_fall_lowMissi.shape)
print(df_param_fall_lowMissi)

param_winter_lowMissi = df_param_winter_lowMissi['Soil_moisture']
print(param_winter_lowMissi.shape)
print(param_winter_lowMissi)

param_spring_lowMissi = df_param_spring_lowMissi['Soil_moisture']
print(param_spring_lowMissi.shape)
print(param_spring_lowMissi)

param_summer_lowMissi = df_param_summer_lowMissi['Soil_moisture']
print(param_summer_lowMissi.shape)
print(param_summer_lowMissi)

param_fall_lowMissi = df_param_fall_lowMissi['Soil_moisture']
print(param_fall_lowMissi.shape)
print(param_fall_lowMissi)



(2052,)
<xarray.DataArray (Month_steps: 2052)>
array([37.22307601, 36.97958712, 36.35035619, ..., 28.85584487,
       32.75164885, 35.88834539])
Coordinates:
  * Month_steps  (Month_steps) int32 0 1 2 3 4 5 ... 2047 2048 2049 2050 2051
(2052,)
<xarray.DataArray (Month_steps: 3)>
array([37.22307601, 36.97958712, 36.35035619])
Coordinates:
  * Month_steps  (Month_steps) datetime64[ns] 1930-01-01 1930-02-01 1930-03-01
(2052, 2)
  Month_steps  Soil_moisture
0  1930-01-01      37.223076
1  1930-02-01      36.979587
2  1930-03-01      36.350356
(2052, 3)
   Month_steps  Soil_moisture  Season_year
0   1930-01-01      37.223076         1930
1   1930-02-01      36.979587         1930
2   1930-03-01      36.350356         1930
3   1930-04-01      35.280689         1930
4   1930-05-01      32.525967         1930
5   1930-06-01      29.076879         1930
6   1930-07-01      26.101310         1930
7   1930-08-01      25.334733         1930
8   1930-09-01      26.912091         1930
9   1930-10-01 

## Plotting Soil moisture Values for the four basins in the four seasons

In [42]:
print(param_winter_Missouri.shape)
print(param_spring_Missouri.shape)
print(param_summer_Missouri.shape)
print(param_fall_Missouri.shape)

print('')
print(param_winter_Arkansas.shape)
print(param_spring_Arkansas.shape)
print(param_summer_Arkansas.shape)
print(param_fall_Arkansas.shape)

print('')
print(param_winter_Ohio.shape)
print(param_spring_Ohio.shape)
print(param_summer_Ohio.shape)
print(param_fall_Ohio.shape)

print('')
print(param_winter_lowMissi.shape)
print(param_spring_lowMissi.shape)
print(param_summer_lowMissi.shape)
print(param_fall_lowMissi.shape)



(171,)
(171,)
(171,)
(171,)

(171,)
(171,)
(171,)
(171,)

(171,)
(171,)
(171,)
(171,)

(171,)
(171,)
(171,)
(171,)


## Find the maximum value for each basin

In [43]:
param_max_Missouri = np.nanmax(param_winter_Missouri)
print(param_max_Missouri)

param_max_Arkansas = np.nanmax(param_winter_Arkansas)
print(param_max_Arkansas)

param_max_Ohio = np.nanmax(param_winter_Ohio)
print(param_max_Ohio)

param_max_lowMissi = np.nanmax(param_winter_lowMissi)
print(param_max_lowMissi)


34.079423526391054
31.46352801322937
42.86311198128594
37.27492418652489


In [44]:

param_winter_ratio_Missouri = param_winter_Missouri/param_max_Missouri
param_spring_ratio_Missouri = param_spring_Missouri/param_max_Missouri
param_summer_ratio_Missouri = param_summer_Missouri/param_max_Missouri
param_fall_ratio_Missouri = param_fall_Missouri/param_max_Missouri
print('')


param_winter_ratio_Arkansas = param_winter_Arkansas/param_max_Arkansas
param_spring_ratio_Arkansas = param_spring_Arkansas/param_max_Arkansas
param_summer_ratio_Arkansas = param_summer_Arkansas/param_max_Arkansas
param_fall_ratio_Arkansas = param_fall_Arkansas/param_max_Arkansas
print('')

param_winter_ratio_Ohio = param_winter_Ohio/param_max_Ohio
param_spring_ratio_Ohio = param_spring_Ohio/param_max_Ohio
param_summer_ratio_Ohio = param_summer_Ohio/param_max_Ohio
param_fall_ratio_Ohio = param_fall_Ohio/param_max_Ohio
print('')

param_winter_ratio_lowMissi = param_winter_lowMissi/param_max_lowMissi
param_spring_ratio_lowMissi = param_spring_lowMissi/param_max_lowMissi
param_summer_ratio_lowMissi = param_summer_lowMissi/param_max_lowMissi
param_fall_ratio_lowMissi = param_fall_lowMissi/param_max_lowMissi
print('')


## Compute Anomalies

## The Four time slots

In [45]:
histYrSelSt = 1930

refPeriodYrSt = 1971
refPeriodYrEnd = 2000

scenYrSt = 2015
scenEraTwoYrSt = 2045
scenEraThreeYrSt = 2075

scenYrEnd = 2100
#

In [46]:
nMonsPerYr = 12     
print(nMonsPerYr)

12


## Reference Period

In [47]:
refPeriodStepSt = refPeriodYrSt - histYrSelSt
refPeriodStepEnd = refPeriodYrEnd - histYrSelSt + 1
print(refPeriodStepSt, refPeriodStepEnd)
print(refPeriodStepEnd-refPeriodStepSt)

41 71
30


## Era1

In [48]:
scenEraOneStepSt = scenYrSt - histYrSelSt
scenEraOneStepEnd = scenEraTwoYrSt - histYrSelSt
print(scenEraOneStepSt, scenEraOneStepEnd)
print(scenEraOneStepEnd-scenEraOneStepSt)

85 115
30


## Era2

In [49]:
scenEraTwoStepSt = scenEraTwoYrSt - histYrSelSt
scenEraTwoStepEnd = scenEraThreeYrSt - histYrSelSt
print(scenEraTwoStepSt, scenEraTwoStepEnd)
print(scenEraTwoStepEnd-scenEraTwoStepSt)

115 145
30


## Era3

In [50]:
scenEraThreeStepSt = scenEraThreeYrSt - histYrSelSt
scenEraThreeStepEnd = scenYrEnd - histYrSelSt + 1
print(scenEraThreeStepSt, scenEraThreeStepEnd)
print(scenEraThreeStepEnd-scenEraThreeStepSt)

145 171
26


In [51]:
print(param_winter_Missouri.shape)
print(param_spring_Missouri.shape)
print(param_summer_Missouri.shape)
print(param_fall_Missouri.shape)

print('')
print(param_winter_Arkansas.shape)
print(param_spring_Arkansas.shape)
print(param_summer_Arkansas.shape)
print(param_fall_Arkansas.shape)

print('')
print(param_winter_Ohio.shape)
print(param_spring_Ohio.shape)
print(param_summer_Ohio.shape)
print(param_fall_Ohio.shape)

print('')
print(param_winter_lowMissi.shape)
print(param_spring_lowMissi.shape)
print(param_summer_lowMissi.shape)
print(param_fall_lowMissi.shape)


(171,)
(171,)
(171,)
(171,)

(171,)
(171,)
(171,)
(171,)

(171,)
(171,)
(171,)
(171,)

(171,)
(171,)
(171,)
(171,)


## Missouri basin

In [52]:
## Winter
param_winter_refPeriod_Missouri = param_winter_Missouri[refPeriodStepSt:refPeriodStepEnd]
print(param_winter_refPeriod_Missouri.shape)

param_winter_eraOne_Missouri = param_winter_Missouri[scenEraOneStepSt:scenEraOneStepEnd]
print(param_winter_eraOne_Missouri.shape)

param_winter_eraTwo_Missouri = param_winter_Missouri[scenEraTwoStepSt:scenEraTwoStepEnd]
print(param_winter_eraTwo_Missouri.shape)

param_winter_eraThree_Missouri = param_winter_Missouri[scenEraThreeStepSt:scenEraThreeStepEnd]
print(param_winter_eraThree_Missouri.shape)
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

## Spring
param_spring_refPeriod_Missouri = param_spring_Missouri[refPeriodStepSt:refPeriodStepEnd]
print(param_spring_refPeriod_Missouri.shape)

param_spring_eraOne_Missouri = param_spring_Missouri[scenEraOneStepSt:scenEraOneStepEnd]
print(param_spring_eraOne_Missouri.shape)

param_spring_eraTwo_Missouri = param_spring_Missouri[scenEraTwoStepSt:scenEraTwoStepEnd]
print(param_spring_eraTwo_Missouri.shape)

param_spring_eraThree_Missouri = param_spring_Missouri[scenEraThreeStepSt:scenEraThreeStepEnd]
print(param_spring_eraThree_Missouri.shape)

## Summer
param_summer_refPeriod_Missouri = param_summer_Missouri[refPeriodStepSt:refPeriodStepEnd]
print(param_summer_refPeriod_Missouri.shape)

param_summer_eraOne_Missouri = param_summer_Missouri[scenEraOneStepSt:scenEraOneStepEnd]
print(param_summer_eraOne_Missouri.shape)

param_summer_eraTwo_Missouri = param_summer_Missouri[scenEraTwoStepSt:scenEraTwoStepEnd]
print(param_summer_eraTwo_Missouri.shape)

param_summer_eraThree_Missouri = param_summer_Missouri[scenEraThreeStepSt:scenEraThreeStepEnd]
print(param_summer_eraThree_Missouri.shape)

## Fall
param_fall_refPeriod_Missouri = param_fall_Missouri[refPeriodStepSt:refPeriodStepEnd]
print(param_fall_refPeriod_Missouri.shape)

param_fall_eraOne_Missouri = param_fall_Missouri[scenEraOneStepSt:scenEraOneStepEnd]
print(param_fall_eraOne_Missouri.shape)

param_fall_eraTwo_Missouri = param_fall_Missouri[scenEraTwoStepSt:scenEraTwoStepEnd]
print(param_fall_eraTwo_Missouri.shape)

param_fall_eraThree_Missouri = param_fall_Missouri[scenEraThreeStepSt:scenEraThreeStepEnd]
print(param_fall_eraThree_Missouri.shape)
#====================================================


(30,)
(30,)
(30,)
(26,)
(30,)
(30,)
(30,)
(26,)
(30,)
(30,)
(30,)
(26,)
(30,)
(30,)
(30,)
(26,)


In [53]:
## Find the mean and standard deviation of the reference period values
## Winter
param_winter_refPeriodMean_refPeriod_Missouri = np.nanmean(param_winter_refPeriod_Missouri)
print(param_winter_refPeriodMean_refPeriod_Missouri)
param_winter_refPeriodStd_refPeriod_Missouri = np.nanstd(param_winter_refPeriod_Missouri)
print(param_winter_refPeriodStd_refPeriod_Missouri)

## Spring
param_spring_refPeriodMean_refPeriod_Missouri = np.nanmean(param_spring_refPeriod_Missouri)
print(param_spring_refPeriodMean_refPeriod_Missouri)
param_spring_refPeriodStd_refPeriod_Missouri = np.nanstd(param_spring_refPeriod_Missouri)
print(param_spring_refPeriodStd_refPeriod_Missouri)

## Summer
param_summer_refPeriodMean_refPeriod_Missouri = np.nanmean(param_summer_refPeriod_Missouri)
print(param_summer_refPeriodMean_refPeriod_Missouri)
param_summer_refPeriodStd_refPeriod_Missouri = np.nanstd(param_summer_refPeriod_Missouri)
print(param_summer_refPeriodStd_refPeriod_Missouri)

## Fall
param_fall_refPeriodMean_refPeriod_Missouri = np.nanmean(param_fall_refPeriod_Missouri)
print(param_fall_refPeriodMean_refPeriod_Missouri)
param_fall_refPeriodStd_refPeriod_Missouri = np.nanstd(param_fall_refPeriod_Missouri)
print(param_fall_refPeriodStd_refPeriod_Missouri)
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

32.47736707306705
0.2430463935983859
32.63526434843881
0.23422593465471142
23.4524905247015
0.3526987294742244
24.120178125598176
0.38163504408469945


In [54]:
## For Anom Plot
####### Winter
param_winterAnom_Missouri = (param_winter_Missouri - param_winter_refPeriodMean_refPeriod_Missouri)
print(param_winterAnom_Missouri.shape)
param_winterAnomScore_Missouri = (param_winter_Missouri - param_winter_refPeriodMean_refPeriod_Missouri)/param_winter_refPeriodStd_refPeriod_Missouri
print(param_winterAnomScore_Missouri.shape)

## For Box Plot
param_winterAnom_eraOne_Missouri = param_winterAnom_Missouri[scenEraOneStepSt:scenEraOneStepEnd]
print(param_winterAnom_eraOne_Missouri.shape)
param_winterAnom_eraTwo_Missouri = param_winterAnom_Missouri[scenEraTwoStepSt:scenEraTwoStepEnd]
print(param_winterAnom_eraTwo_Missouri.shape)
param_winterAnom_eraThree_Missouri = param_winterAnom_Missouri[scenEraThreeStepSt:scenEraThreeStepEnd]
print(param_winterAnom_eraThree_Missouri.shape)
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

####### Spring
param_springAnom_Missouri = (param_spring_Missouri - param_spring_refPeriodMean_refPeriod_Missouri)
print(param_springAnom_Missouri.shape)
param_springAnomScore_Missouri = (param_spring_Missouri - param_spring_refPeriodMean_refPeriod_Missouri)/param_spring_refPeriodStd_refPeriod_Missouri
print(param_springAnomScore_Missouri.shape)

## For Box Plot
param_springAnom_eraOne_Missouri = param_springAnom_Missouri[scenEraOneStepSt:scenEraOneStepEnd]
print(param_springAnom_eraOne_Missouri.shape)
param_springAnom_eraTwo_Missouri = param_springAnom_Missouri[scenEraTwoStepSt:scenEraTwoStepEnd]
print(param_springAnom_eraTwo_Missouri.shape)
param_springAnom_eraThree_Missouri = param_springAnom_Missouri[scenEraThreeStepSt:scenEraThreeStepEnd]
print(param_springAnom_eraThree_Missouri.shape)
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

####### Summer
param_summerAnom_Missouri = (param_summer_Missouri - param_summer_refPeriodMean_refPeriod_Missouri)
print(param_summerAnom_Missouri.shape)
param_summerAnomScore_Missouri = (param_summer_Missouri - param_summer_refPeriodMean_refPeriod_Missouri)/param_summer_refPeriodStd_refPeriod_Missouri
print(param_summerAnomScore_Missouri.shape)

## For Box Plot
param_summerAnom_eraOne_Missouri = param_summerAnom_Missouri[scenEraOneStepSt:scenEraOneStepEnd]
print(param_summerAnom_eraOne_Missouri.shape)
param_summerAnom_eraTwo_Missouri = param_summerAnom_Missouri[scenEraTwoStepSt:scenEraTwoStepEnd]
print(param_summerAnom_eraTwo_Missouri.shape)
param_summerAnom_eraThree_Missouri = param_summerAnom_Missouri[scenEraThreeStepSt:scenEraThreeStepEnd]
print(param_summerAnom_eraThree_Missouri.shape)
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

####### Fall
param_fallAnom_Missouri = (param_fall_Missouri - param_fall_refPeriodMean_refPeriod_Missouri)
print(param_fallAnom_Missouri.shape)
param_fallAnomScore_Missouri = (param_fall_Missouri - param_fall_refPeriodMean_refPeriod_Missouri)/param_fall_refPeriodStd_refPeriod_Missouri
print(param_fallAnomScore_Missouri.shape)

## For Box Plot
param_fallAnom_eraOne_Missouri = param_fallAnom_Missouri[scenEraOneStepSt:scenEraOneStepEnd]
print(param_fallAnom_eraOne_Missouri.shape)
param_fallAnom_eraTwo_Missouri = param_fallAnom_Missouri[scenEraTwoStepSt:scenEraTwoStepEnd]
print(param_fallAnom_eraTwo_Missouri.shape)
param_fallAnom_eraThree_Missouri = param_fallAnom_Missouri[scenEraThreeStepSt:scenEraThreeStepEnd]
print(param_fallAnom_eraThree_Missouri.shape)
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++



(171,)
(171,)
(30,)
(30,)
(26,)
(171,)
(171,)
(30,)
(30,)
(26,)
(171,)
(171,)
(30,)
(30,)
(26,)
(171,)
(171,)
(30,)
(30,)
(26,)


## Arkansas basin

In [55]:
## Winter
param_winter_refPeriod_Arkansas = param_winter_Arkansas[refPeriodStepSt:refPeriodStepEnd]
print(param_winter_refPeriod_Arkansas.shape)

param_winter_eraOne_Arkansas = param_winter_Arkansas[scenEraOneStepSt:scenEraOneStepEnd]
print(param_winter_eraOne_Arkansas.shape)

param_winter_eraTwo_Arkansas = param_winter_Arkansas[scenEraTwoStepSt:scenEraTwoStepEnd]
print(param_winter_eraTwo_Arkansas.shape)

param_winter_eraThree_Arkansas = param_winter_Arkansas[scenEraThreeStepSt:scenEraThreeStepEnd]
print(param_winter_eraThree_Arkansas.shape)
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

## Spring
param_spring_refPeriod_Arkansas = param_spring_Arkansas[refPeriodStepSt:refPeriodStepEnd]
print(param_spring_refPeriod_Arkansas.shape)

param_spring_eraOne_Arkansas = param_spring_Arkansas[scenEraOneStepSt:scenEraOneStepEnd]
print(param_spring_eraOne_Arkansas.shape)

param_spring_eraTwo_Arkansas = param_spring_Arkansas[scenEraTwoStepSt:scenEraTwoStepEnd]
print(param_spring_eraTwo_Arkansas.shape)

param_spring_eraThree_Arkansas = param_spring_Arkansas[scenEraThreeStepSt:scenEraThreeStepEnd]
print(param_spring_eraThree_Arkansas.shape)

## Summer
param_summer_refPeriod_Arkansas = param_summer_Arkansas[refPeriodStepSt:refPeriodStepEnd]
print(param_summer_refPeriod_Arkansas.shape)

param_summer_eraOne_Arkansas = param_summer_Arkansas[scenEraOneStepSt:scenEraOneStepEnd]
print(param_summer_eraOne_Arkansas.shape)

param_summer_eraTwo_Arkansas = param_summer_Arkansas[scenEraTwoStepSt:scenEraTwoStepEnd]
print(param_summer_eraTwo_Arkansas.shape)

param_summer_eraThree_Arkansas = param_summer_Arkansas[scenEraThreeStepSt:scenEraThreeStepEnd]
print(param_summer_eraThree_Arkansas.shape)

## Fall
param_fall_refPeriod_Arkansas = param_fall_Arkansas[refPeriodStepSt:refPeriodStepEnd]
print(param_fall_refPeriod_Arkansas.shape)

param_fall_eraOne_Arkansas = param_fall_Arkansas[scenEraOneStepSt:scenEraOneStepEnd]
print(param_fall_eraOne_Arkansas.shape)

param_fall_eraTwo_Arkansas = param_fall_Arkansas[scenEraTwoStepSt:scenEraTwoStepEnd]
print(param_fall_eraTwo_Arkansas.shape)

param_fall_eraThree_Arkansas = param_fall_Arkansas[scenEraThreeStepSt:scenEraThreeStepEnd]
print(param_fall_eraThree_Arkansas.shape)
#====================================================


(30,)
(30,)
(30,)
(26,)
(30,)
(30,)
(30,)
(26,)
(30,)
(30,)
(30,)
(26,)
(30,)
(30,)
(30,)
(26,)


In [56]:
## Find the mean and standard deviation of the reference period values
## Winter
param_winter_refPeriodMean_refPeriod_Arkansas = np.nanmean(param_winter_refPeriod_Arkansas)
print(param_winter_refPeriodMean_refPeriod_Arkansas)
param_winter_refPeriodStd_refPeriod_Arkansas = np.nanstd(param_winter_refPeriod_Arkansas)
print(param_winter_refPeriodStd_refPeriod_Arkansas)

## Spring
param_spring_refPeriodMean_refPeriod_Arkansas = np.nanmean(param_spring_refPeriod_Arkansas)
print(param_spring_refPeriodMean_refPeriod_Arkansas)
param_spring_refPeriodStd_refPeriod_Arkansas = np.nanstd(param_spring_refPeriod_Arkansas)
print(param_spring_refPeriodStd_refPeriod_Arkansas)

## Summer
param_summer_refPeriodMean_refPeriod_Arkansas = np.nanmean(param_summer_refPeriod_Arkansas)
print(param_summer_refPeriodMean_refPeriod_Arkansas)
param_summer_refPeriodStd_refPeriod_Arkansas = np.nanstd(param_summer_refPeriod_Arkansas)
print(param_summer_refPeriodStd_refPeriod_Arkansas)

## Fall
param_fall_refPeriodMean_refPeriod_Arkansas = np.nanmean(param_fall_refPeriod_Arkansas)
print(param_fall_refPeriodMean_refPeriod_Arkansas)
param_fall_refPeriodStd_refPeriod_Arkansas = np.nanstd(param_fall_refPeriod_Arkansas)
print(param_fall_refPeriodStd_refPeriod_Arkansas)
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

30.31884008271017
0.35658390306243337
28.645837183151723
0.3569472418679746
20.45941270117112
0.37742237360043945
23.666446768949065
0.38733385599408054


In [57]:
## For Anom Plot
####### Winter
param_winterAnom_Arkansas = (param_winter_Arkansas - param_winter_refPeriodMean_refPeriod_Arkansas)
print(param_winterAnom_Arkansas.shape)
param_winterAnomScore_Arkansas = (param_winter_Arkansas - param_winter_refPeriodMean_refPeriod_Arkansas)/param_winter_refPeriodStd_refPeriod_Arkansas
print(param_winterAnomScore_Arkansas.shape)

## For Box Plot
param_winterAnom_eraOne_Arkansas = param_winterAnom_Arkansas[scenEraOneStepSt:scenEraOneStepEnd]
print(param_winterAnom_eraOne_Arkansas.shape)
param_winterAnom_eraTwo_Arkansas = param_winterAnom_Arkansas[scenEraTwoStepSt:scenEraTwoStepEnd]
print(param_winterAnom_eraTwo_Arkansas.shape)
param_winterAnom_eraThree_Arkansas = param_winterAnom_Arkansas[scenEraThreeStepSt:scenEraThreeStepEnd]
print(param_winterAnom_eraThree_Arkansas.shape)
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

##### Spring
param_springAnom_Arkansas = (param_spring_Arkansas - param_spring_refPeriodMean_refPeriod_Arkansas)
print(param_springAnom_Arkansas.shape)
param_springAnomScore_Arkansas = (param_spring_Arkansas - param_spring_refPeriodMean_refPeriod_Arkansas)/param_spring_refPeriodStd_refPeriod_Arkansas
print(param_springAnomScore_Arkansas.shape)

## For Box Plot
param_springAnom_eraOne_Arkansas = param_springAnom_Arkansas[scenEraOneStepSt:scenEraOneStepEnd]
print(param_springAnom_eraOne_Arkansas.shape)
param_springAnom_eraTwo_Arkansas = param_springAnom_Arkansas[scenEraTwoStepSt:scenEraTwoStepEnd]
print(param_springAnom_eraTwo_Arkansas.shape)
param_springAnom_eraThree_Arkansas = param_springAnom_Arkansas[scenEraThreeStepSt:scenEraThreeStepEnd]
print(param_springAnom_eraThree_Arkansas.shape)
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

####### Summer
param_summerAnom_Arkansas = (param_summer_Arkansas - param_summer_refPeriodMean_refPeriod_Arkansas)
print(param_summerAnom_Arkansas.shape)
param_summerAnomScore_Arkansas = (param_summer_Arkansas - param_summer_refPeriodMean_refPeriod_Arkansas)/param_summer_refPeriodStd_refPeriod_Arkansas
print(param_summerAnomScore_Arkansas.shape)

## For Box Plot
param_summerAnom_eraOne_Arkansas = param_summerAnom_Arkansas[scenEraOneStepSt:scenEraOneStepEnd]
print(param_summerAnom_eraOne_Arkansas.shape)
param_summerAnom_eraTwo_Arkansas = param_summerAnom_Arkansas[scenEraTwoStepSt:scenEraTwoStepEnd]
print(param_summerAnom_eraTwo_Arkansas.shape)
param_summerAnom_eraThree_Arkansas = param_summerAnom_Arkansas[scenEraThreeStepSt:scenEraThreeStepEnd]
print(param_summerAnom_eraThree_Arkansas.shape)
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

####### Fall
param_fallAnom_Arkansas = (param_fall_Arkansas - param_fall_refPeriodMean_refPeriod_Arkansas)
print(param_fallAnom_Arkansas.shape)
param_fallAnomScore_Arkansas = (param_fall_Arkansas - param_fall_refPeriodMean_refPeriod_Arkansas)/param_fall_refPeriodStd_refPeriod_Arkansas
print(param_fallAnomScore_Arkansas.shape)

## For Box Plot
param_fallAnom_eraOne_Arkansas = param_fallAnom_Arkansas[scenEraOneStepSt:scenEraOneStepEnd]
print(param_fallAnom_eraOne_Arkansas.shape)
param_fallAnom_eraTwo_Arkansas = param_fallAnom_Arkansas[scenEraTwoStepSt:scenEraTwoStepEnd]
print(param_fallAnom_eraTwo_Arkansas.shape)
param_fallAnom_eraThree_Arkansas = param_fallAnom_Arkansas[scenEraThreeStepSt:scenEraThreeStepEnd]
print(param_fallAnom_eraThree_Arkansas.shape)
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++



(171,)
(171,)
(30,)
(30,)
(26,)
(171,)
(171,)
(30,)
(30,)
(26,)
(171,)
(171,)
(30,)
(30,)
(26,)
(171,)
(171,)
(30,)
(30,)
(26,)


## Ohio basin

In [58]:
## Winter
param_winter_refPeriod_Ohio = param_winter_Ohio[refPeriodStepSt:refPeriodStepEnd]
print(param_winter_refPeriod_Ohio.shape)

param_winter_eraOne_Ohio = param_winter_Ohio[scenEraOneStepSt:scenEraOneStepEnd]
print(param_winter_eraOne_Ohio.shape)

param_winter_eraTwo_Ohio = param_winter_Ohio[scenEraTwoStepSt:scenEraTwoStepEnd]
print(param_winter_eraTwo_Ohio.shape)

param_winter_eraThree_Ohio = param_winter_Ohio[scenEraThreeStepSt:scenEraThreeStepEnd]
print(param_winter_eraThree_Ohio.shape)
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

## Spring
param_spring_refPeriod_Ohio = param_spring_Ohio[refPeriodStepSt:refPeriodStepEnd]
print(param_spring_refPeriod_Ohio.shape)

param_spring_eraOne_Ohio = param_spring_Ohio[scenEraOneStepSt:scenEraOneStepEnd]
print(param_spring_eraOne_Ohio.shape)

param_spring_eraTwo_Ohio = param_spring_Ohio[scenEraTwoStepSt:scenEraTwoStepEnd]
print(param_spring_eraTwo_Ohio.shape)

param_spring_eraThree_Ohio = param_spring_Ohio[scenEraThreeStepSt:scenEraThreeStepEnd]
print(param_spring_eraThree_Ohio.shape)

## Summer
param_summer_refPeriod_Ohio = param_summer_Ohio[refPeriodStepSt:refPeriodStepEnd]
print(param_summer_refPeriod_Ohio.shape)

param_summer_eraOne_Ohio = param_summer_Ohio[scenEraOneStepSt:scenEraOneStepEnd]
print(param_summer_eraOne_Ohio.shape)

param_summer_eraTwo_Ohio = param_summer_Ohio[scenEraTwoStepSt:scenEraTwoStepEnd]
print(param_summer_eraTwo_Ohio.shape)

param_summer_eraThree_Ohio = param_summer_Ohio[scenEraThreeStepSt:scenEraThreeStepEnd]
print(param_summer_eraThree_Ohio.shape)

## Fall
param_fall_refPeriod_Ohio = param_fall_Ohio[refPeriodStepSt:refPeriodStepEnd]
print(param_fall_refPeriod_Ohio.shape)

param_fall_eraOne_Ohio = param_fall_Ohio[scenEraOneStepSt:scenEraOneStepEnd]
print(param_fall_eraOne_Ohio.shape)

param_fall_eraTwo_Ohio = param_fall_Ohio[scenEraTwoStepSt:scenEraTwoStepEnd]
print(param_fall_eraTwo_Ohio.shape)

param_fall_eraThree_Ohio = param_fall_Ohio[scenEraThreeStepSt:scenEraThreeStepEnd]
print(param_fall_eraThree_Ohio.shape)
#====================================================


(30,)
(30,)
(30,)
(26,)
(30,)
(30,)
(30,)
(26,)
(30,)
(30,)
(30,)
(26,)
(30,)
(30,)
(30,)
(26,)


In [59]:
## Find the mean and standard deviation of the reference period values
## Winter
param_winter_refPeriodMean_refPeriod_Ohio = np.nanmean(param_winter_refPeriod_Ohio)
print(param_winter_refPeriodMean_refPeriod_Ohio)
param_winter_refPeriodStd_refPeriod_Ohio = np.nanstd(param_winter_refPeriod_Ohio)
print(param_winter_refPeriodStd_refPeriod_Ohio)

## Spring
param_spring_refPeriodMean_refPeriod_Ohio = np.nanmean(param_spring_refPeriod_Ohio)
print(param_spring_refPeriodMean_refPeriod_Ohio)
param_spring_refPeriodStd_refPeriod_Ohio = np.nanstd(param_spring_refPeriod_Ohio)
print(param_spring_refPeriodStd_refPeriod_Ohio)

## Summer
param_summer_refPeriodMean_refPeriod_Ohio = np.nanmean(param_summer_refPeriod_Ohio)
print(param_summer_refPeriodMean_refPeriod_Ohio)
param_summer_refPeriodStd_refPeriod_Ohio = np.nanstd(param_summer_refPeriod_Ohio)
print(param_summer_refPeriodStd_refPeriod_Ohio)

## Fall
param_fall_refPeriodMean_refPeriod_Ohio = np.nanmean(param_fall_refPeriod_Ohio)
print(param_fall_refPeriodMean_refPeriod_Ohio)
param_fall_refPeriodStd_refPeriod_Ohio = np.nanstd(param_fall_refPeriod_Ohio)
print(param_fall_refPeriodStd_refPeriod_Ohio)
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

41.16137874600445
0.44436920657030093
32.63526434843881
0.23422593465471142
23.4524905247015
0.3526987294742244
29.076392031153926
0.5344839517826648


In [60]:
## For Anom Plot
####### Winter
param_winterAnom_Ohio = (param_winter_Ohio - param_winter_refPeriodMean_refPeriod_Ohio)
print(param_winterAnom_Ohio.shape)
param_winterAnomScore_Ohio = (param_winter_Ohio - param_winter_refPeriodMean_refPeriod_Ohio)/param_winter_refPeriodStd_refPeriod_Ohio
print(param_winterAnomScore_Ohio.shape)

## For Box Plot
param_winterAnom_eraOne_Ohio = param_winterAnom_Ohio[scenEraOneStepSt:scenEraOneStepEnd]
print(param_winterAnom_eraOne_Ohio.shape)
param_winterAnom_eraTwo_Ohio = param_winterAnom_Ohio[scenEraTwoStepSt:scenEraTwoStepEnd]
print(param_winterAnom_eraTwo_Ohio.shape)
param_winterAnom_eraThree_Ohio = param_winterAnom_Ohio[scenEraThreeStepSt:scenEraThreeStepEnd]
print(param_winterAnom_eraThree_Ohio.shape)
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

####### Spring
param_springAnom_Ohio = (param_spring_Ohio - param_spring_refPeriodMean_refPeriod_Ohio)
print(param_springAnom_Ohio.shape)
param_springAnomScore_Ohio = (param_spring_Ohio - param_spring_refPeriodMean_refPeriod_Ohio)/param_spring_refPeriodStd_refPeriod_Ohio
print(param_springAnomScore_Ohio.shape)

## For Box Plot
param_springAnom_eraOne_Ohio = param_springAnom_Ohio[scenEraOneStepSt:scenEraOneStepEnd]
print(param_springAnom_eraOne_Ohio.shape)
param_springAnom_eraTwo_Ohio = param_springAnom_Ohio[scenEraTwoStepSt:scenEraTwoStepEnd]
print(param_springAnom_eraTwo_Ohio.shape)
param_springAnom_eraThree_Ohio = param_springAnom_Ohio[scenEraThreeStepSt:scenEraThreeStepEnd]
print(param_springAnom_eraThree_Ohio.shape)
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

####### Summer
param_summerAnom_Ohio = (param_summer_Ohio - param_summer_refPeriodMean_refPeriod_Ohio)
print(param_summerAnom_Ohio.shape)
param_summerAnomScore_Ohio = (param_summer_Ohio - param_summer_refPeriodMean_refPeriod_Ohio)/param_summer_refPeriodStd_refPeriod_Ohio
print(param_summerAnomScore_Ohio.shape)

## For Box Plot
param_summerAnom_eraOne_Ohio = param_summerAnom_Ohio[scenEraOneStepSt:scenEraOneStepEnd]
print(param_summerAnom_eraOne_Ohio.shape)
param_summerAnom_eraTwo_Ohio = param_summerAnom_Ohio[scenEraTwoStepSt:scenEraTwoStepEnd]
print(param_summerAnom_eraTwo_Ohio.shape)
param_summerAnom_eraThree_Ohio = param_summerAnom_Ohio[scenEraThreeStepSt:scenEraThreeStepEnd]
print(param_summerAnom_eraThree_Ohio.shape)
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

####### Fall
param_fallAnom_Ohio = (param_fall_Ohio - param_fall_refPeriodMean_refPeriod_Ohio)
print(param_fallAnom_Ohio.shape)
param_fallAnomScore_Ohio = (param_fall_Ohio - param_fall_refPeriodMean_refPeriod_Ohio)/param_fall_refPeriodStd_refPeriod_Ohio
print(param_fallAnomScore_Ohio.shape)

## For Box Plot
param_fallAnom_eraOne_Ohio = param_fallAnom_Ohio[scenEraOneStepSt:scenEraOneStepEnd]
print(param_fallAnom_eraOne_Ohio.shape)
param_fallAnom_eraTwo_Ohio = param_fallAnom_Ohio[scenEraTwoStepSt:scenEraTwoStepEnd]
print(param_fallAnom_eraTwo_Ohio.shape)
param_fallAnom_eraThree_Ohio = param_fallAnom_Ohio[scenEraThreeStepSt:scenEraThreeStepEnd]
print(param_fallAnom_eraThree_Ohio.shape)
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

(171,)
(171,)
(30,)
(30,)
(26,)
(171,)
(171,)
(30,)
(30,)
(26,)
(171,)
(171,)
(30,)
(30,)
(26,)
(171,)
(171,)
(30,)
(30,)
(26,)


## Lower Mississippi River Basin

In [61]:
## Winter
param_winter_refPeriod_lowMissi = param_winter_lowMissi[refPeriodStepSt:refPeriodStepEnd]
print(param_winter_refPeriod_lowMissi.shape)

param_winter_eraOne_lowMissi = param_winter_lowMissi[scenEraOneStepSt:scenEraOneStepEnd]
print(param_winter_eraOne_lowMissi.shape)

param_winter_eraTwo_lowMissi = param_winter_lowMissi[scenEraTwoStepSt:scenEraTwoStepEnd]
print(param_winter_eraTwo_lowMissi.shape)

param_winter_eraThree_lowMissi = param_winter_lowMissi[scenEraThreeStepSt:scenEraThreeStepEnd]
print(param_winter_eraThree_lowMissi.shape)
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

## Spring
param_spring_refPeriod_lowMissi = param_spring_lowMissi[refPeriodStepSt:refPeriodStepEnd]
print(param_spring_refPeriod_lowMissi.shape)

param_spring_eraOne_lowMissi = param_spring_lowMissi[scenEraOneStepSt:scenEraOneStepEnd]
print(param_spring_eraOne_lowMissi.shape)

param_spring_eraTwo_lowMissi = param_spring_lowMissi[scenEraTwoStepSt:scenEraTwoStepEnd]
print(param_spring_eraTwo_lowMissi.shape)

param_spring_eraThree_lowMissi = param_spring_lowMissi[scenEraThreeStepSt:scenEraThreeStepEnd]
print(param_spring_eraThree_lowMissi.shape)

## Summer
param_summer_refPeriod_lowMissi = param_summer_lowMissi[refPeriodStepSt:refPeriodStepEnd]
print(param_summer_refPeriod_lowMissi.shape)

param_summer_eraOne_lowMissi = param_summer_lowMissi[scenEraOneStepSt:scenEraOneStepEnd]
print(param_summer_eraOne_lowMissi.shape)

param_summer_eraTwo_lowMissi = param_summer_lowMissi[scenEraTwoStepSt:scenEraTwoStepEnd]
print(param_summer_eraTwo_lowMissi.shape)

param_summer_eraThree_lowMissi = param_summer_lowMissi[scenEraThreeStepSt:scenEraThreeStepEnd]
print(param_summer_eraThree_lowMissi.shape)

## Fall
param_fall_refPeriod_lowMissi = param_fall_lowMissi[refPeriodStepSt:refPeriodStepEnd]
print(param_fall_refPeriod_lowMissi.shape)

param_fall_eraOne_lowMissi = param_fall_lowMissi[scenEraOneStepSt:scenEraOneStepEnd]
print(param_fall_eraOne_lowMissi.shape)

param_fall_eraTwo_lowMissi = param_fall_lowMissi[scenEraTwoStepSt:scenEraTwoStepEnd]
print(param_fall_eraTwo_lowMissi.shape)

param_fall_eraThree_lowMissi = param_fall_lowMissi[scenEraThreeStepSt:scenEraThreeStepEnd]
print(param_fall_eraThree_lowMissi.shape)
#====================================================


(30,)
(30,)
(30,)
(26,)
(30,)
(30,)
(30,)
(26,)
(30,)
(30,)
(30,)
(26,)
(30,)
(30,)
(30,)
(26,)


In [62]:
## Find the mean and standard deviation of the reference period values
## Winter
param_winter_refPeriodMean_refPeriod_lowMissi = np.nanmean(param_winter_refPeriod_lowMissi)
print(param_winter_refPeriodMean_refPeriod_lowMissi)
param_winter_refPeriodStd_refPeriod_lowMissi = np.nanstd(param_winter_refPeriod_lowMissi)
print(param_winter_refPeriodStd_refPeriod_lowMissi)

## Spring
param_spring_refPeriodMean_refPeriod_lowMissi = np.nanmean(param_spring_refPeriod_lowMissi)
print(param_spring_refPeriodMean_refPeriod_lowMissi)
param_spring_refPeriodStd_refPeriod_lowMissi = np.nanstd(param_spring_refPeriod_lowMissi)
print(param_spring_refPeriodStd_refPeriod_lowMissi)

## Summer
param_summer_refPeriodMean_refPeriod_lowMissi = np.nanmean(param_summer_refPeriod_lowMissi)
print(param_summer_refPeriodMean_refPeriod_lowMissi)
param_summer_refPeriodStd_refPeriod_lowMissi = np.nanstd(param_summer_refPeriod_lowMissi)
print(param_summer_refPeriodStd_refPeriod_lowMissi)

## Fall
param_fall_refPeriodMean_refPeriod_lowMissi = np.nanmean(param_fall_refPeriod_lowMissi)
print(param_fall_refPeriodMean_refPeriod_lowMissi)
param_fall_refPeriodStd_refPeriod_lowMissi = np.nanstd(param_fall_refPeriod_lowMissi)
print(param_fall_refPeriodStd_refPeriod_lowMissi)
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

36.95006629747057
0.14812252492211994
34.93986370733049
0.15863760095318957
28.258045433241225
0.3455442020563441
30.654987818611993
0.4261780148810915


In [63]:
## For Anom Plot
####### Winter
param_winterAnom_lowMissi = (param_winter_lowMissi - param_winter_refPeriodMean_refPeriod_lowMissi)
print(param_winterAnom_lowMissi.shape)
param_winterAnomScore_lowMissi = (param_winter_lowMissi - param_winter_refPeriodMean_refPeriod_lowMissi)/param_winter_refPeriodStd_refPeriod_lowMissi
print(param_winterAnomScore_lowMissi.shape)

## For Box Plot
param_winterAnom_eraOne_lowMissi = param_winterAnom_lowMissi[scenEraOneStepSt:scenEraOneStepEnd]
print(param_winterAnom_eraOne_lowMissi.shape)
param_winterAnom_eraTwo_lowMissi = param_winterAnom_lowMissi[scenEraTwoStepSt:scenEraTwoStepEnd]
print(param_winterAnom_eraTwo_lowMissi.shape)
param_winterAnom_eraThree_lowMissi = param_winterAnom_lowMissi[scenEraThreeStepSt:scenEraThreeStepEnd]
print(param_winterAnom_eraThree_lowMissi.shape)
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

####### Spring
param_springAnom_lowMissi = (param_spring_lowMissi - param_spring_refPeriodMean_refPeriod_lowMissi)
print(param_springAnom_lowMissi.shape)
param_springAnomScore_lowMissi = (param_spring_lowMissi - param_spring_refPeriodMean_refPeriod_lowMissi)/param_spring_refPeriodStd_refPeriod_lowMissi
print(param_springAnomScore_lowMissi.shape)

## For Box Plot
param_springAnom_eraOne_lowMissi = param_springAnom_lowMissi[scenEraOneStepSt:scenEraOneStepEnd]
print(param_springAnom_eraOne_lowMissi.shape)
param_springAnom_eraTwo_lowMissi = param_springAnom_lowMissi[scenEraTwoStepSt:scenEraTwoStepEnd]
print(param_springAnom_eraTwo_lowMissi.shape)
param_springAnom_eraThree_lowMissi = param_springAnom_lowMissi[scenEraThreeStepSt:scenEraThreeStepEnd]
print(param_springAnom_eraThree_lowMissi.shape)
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

####### Summer
param_summerAnom_lowMissi = (param_summer_lowMissi - param_summer_refPeriodMean_refPeriod_lowMissi)
print(param_summerAnom_lowMissi.shape)
param_summerAnomScore_lowMissi = (param_summer_lowMissi - param_summer_refPeriodMean_refPeriod_lowMissi)/param_summer_refPeriodStd_refPeriod_lowMissi
print(param_summerAnomScore_lowMissi.shape)

## For Box Plot
param_summerAnom_eraOne_lowMissi = param_summerAnom_lowMissi[scenEraOneStepSt:scenEraOneStepEnd]
print(param_summerAnom_eraOne_lowMissi.shape)
param_summerAnom_eraTwo_lowMissi = param_summerAnom_lowMissi[scenEraTwoStepSt:scenEraTwoStepEnd]
print(param_summerAnom_eraTwo_lowMissi.shape)
param_summerAnom_eraThree_lowMissi = param_summerAnom_lowMissi[scenEraThreeStepSt:scenEraThreeStepEnd]
print(param_summerAnom_eraThree_lowMissi.shape)
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

####### Fall
param_fallAnom_lowMissi = (param_fall_lowMissi - param_fall_refPeriodMean_refPeriod_lowMissi)
print(param_fallAnom_lowMissi.shape)
param_fallAnomScore_lowMissi = (param_fall_lowMissi - param_fall_refPeriodMean_refPeriod_lowMissi)/param_fall_refPeriodStd_refPeriod_lowMissi
print(param_fallAnomScore_lowMissi.shape)

## For Box Plot
param_fallAnom_eraOne_lowMissi = param_fallAnom_lowMissi[scenEraOneStepSt:scenEraOneStepEnd]
print(param_fallAnom_eraOne_lowMissi.shape)
param_fallAnom_eraTwo_lowMissi = param_fallAnom_lowMissi[scenEraTwoStepSt:scenEraTwoStepEnd]
print(param_fallAnom_eraTwo_lowMissi.shape)
param_fallAnom_eraThree_lowMissi = param_fallAnom_lowMissi[scenEraThreeStepSt:scenEraThreeStepEnd]
print(param_fallAnom_eraThree_lowMissi.shape)
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


(171,)
(171,)
(30,)
(30,)
(26,)
(171,)
(171,)
(30,)
(30,)
(26,)
(171,)
(171,)
(30,)
(30,)
(26,)
(171,)
(171,)
(30,)
(30,)
(26,)


## Formatting Figure

In [64]:
yrSt = 1930
yrEnd = 2100

timeList = []
for yr in range(yrSt, yrEnd+1):    
    
    
    yMD = datetime.datetime(yr,1,1)    
    timeList.append(yMD)
                   
                        
timeListDf = pd.DataFrame(timeList)

timeListDf.columns = ['t1']
timeListDf


,t1
0,1930-01-01
1,1931-01-01
2,1932-01-01
3,1933-01-01
4,1934-01-01
...,...
166,2096-01-01
167,2097-01-01
168,2098-01-01
169,2099-01-01


In [65]:
yMD = timeListDf['t1'].dt.strftime('%Y').sort_values()
multiple = 20
multiple = np.ceil(multiple)
timeTickLebels = yMD[yMD.index % multiple == 0]
timeTicks = yMD.index[yMD.index % multiple == 0]
timeTicks


Int64Index([0, 20, 40, 60, 80, 100, 120, 140, 160], dtype='int64')

In [66]:
width = 0.5  # the width of the bars

my_colors = ["green", "blue", 
             "orange", "red"]
sns.set_palette( my_colors )

<ipython-input-67-8f227662fb19>:29: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax1.xaxis.set_major_formatter(plt.FixedFormatter(yMD))
<ipython-input-67-8f227662fb19>:49: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax2.xaxis.set_major_formatter(plt.FixedFormatter(yMD))
<ipython-input-67-8f227662fb19>:75: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax3.xaxis.set_major_formatter(plt.FixedFormatter(yMD))
<ipython-input-67-8f227662fb19>:100: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax4.xaxis.set_major_formatter(plt.FixedFormatter(yMD))


In [67]:
#%matplotlib notebook


tickFontSize = 14
textFontSize = 16
legendFontSize = 15
titleFontSize = 16

linewidth = 0.8
dashLinewidth = 0.7


#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
fig = plt.figure(figsize=(12, 10), constrained_layout=False)
axs = [plt.subplot(4, 1, x + 1,) for x in range(4)]

ax1 = axs[0]

ln12=ax1.plot(range(171), param_winterAnom_Missouri, label="Winter", color = 'green', linestyle='-', linewidth=1.0)
ln13=ax1.plot(range(171), param_springAnom_Missouri, label="Spring", color = 'blue', linestyle='-', linewidth=1.0)
ln14=ax1.plot(range(171), param_summerAnom_Missouri, label="Summer", color = 'orange', linestyle='-', linewidth=1.0)
ln15=ax1.plot(range(171), param_fallAnom_Missouri, label="Fall", color = 'red', linestyle='-', linewidth=1.0)
ln16=ax1.axhline(y = 0, color = 'k', label = '', linestyle='--', linewidth=0.7)

ln17=ax1.axvspan(1, 84.5, color='b',alpha=0.03)

ax1.set_title('(a) Missouri', y=0.98, pad=-14, fontsize = textFontSize, loc='left')
#+++++++++++++++++++++++++++
ax1.xaxis.set_major_formatter(plt.FixedFormatter(yMD))
ax1.set_xticks(timeTicks)
ax1.set_xticklabels('', )

ax1.set_xlim([0, 170])
ax1.set_ylim([-2.8, 2.0])

ax1.set_yticks([-2, 0]) 
ax1.set_yticklabels(['-2', '0'],  fontsize = tickFontSize) 
#+++++++++++++++++++++++++++++++++++++

ax2 = axs[1]
ln22=ax2.plot(range(171), param_winterAnom_Arkansas, label="Winter", color = 'green', linestyle='-', linewidth=1.0)
ln23=ax2.plot(range(171), param_springAnom_Arkansas, label="Spring", color = 'blue', linestyle='-', linewidth=1.0)
ln24=ax2.plot(range(171), param_summerAnom_Arkansas, label="Summer", color = 'orange', linestyle='-', linewidth=1.0)
ln25=ax2.plot(range(171), param_fallAnom_Arkansas, label="Fall", color = 'red', linestyle='-', linewidth=1.0)
ln26=ax2.axhline(y = 0, color = 'k', label = '', linestyle='--', linewidth=0.7)
ln27=ax2.axvspan(1, 84.5, color='b',alpha=0.03)

ax2.set_title('(b) Arkansas', y=0.98, pad=-14, fontsize = textFontSize, loc='left')
ax2.xaxis.set_major_formatter(plt.FixedFormatter(yMD))
ax2.set_xticks(timeTicks)
ax2.set_xticklabels('', )

ax2.set_xlim([0, 170])
ax2.set_ylim([-2.5, 1.65])

ax2.set_yticks([-2, -1, 0, 1]) 
ax2.set_yticklabels(['-2', '1', '0', '1'],  fontsize = tickFontSize) 

ax2.set_ylabel('Anomaly, kg/$\mathregular{m^{2}}$', fontsize = textFontSize )

ax2.yaxis.set_label_coords(-.045, -.1)

#+++++++++++++++++++++++++++++++++++++
#+++++++++++++++++++++++++++++++++++++

ax3 = axs[2]
ln32=ax3.plot(range(171), param_winterAnom_Ohio, label="Winter", color = 'green', linestyle='-', linewidth=1.0)
ln33=ax3.plot(range(171), param_springAnom_Ohio, label="Spring", color = 'blue', linestyle='-', linewidth=1.0)
ln34=ax3.plot(range(171), param_summerAnom_Ohio, label="Summer", color = 'orange', linestyle='-', linewidth=1.0)
ln35=ax3.plot(range(171), param_fallAnom_Ohio, label="Fall", color = 'red', linestyle='-', linewidth=1.0)
ln36=ax3.axhline(y = 0, color = 'k', label = '', linestyle='--', linewidth=0.7)
ln37=ax3.axvspan(1, 84.5, color='b',alpha=0.03)

ax3.set_title('(c) Ohio', y=0.98, pad=-14, fontsize = textFontSize, loc='left')
ax3.xaxis.set_major_formatter(plt.FixedFormatter(yMD))
ax3.set_xticks(timeTicks)
ax3.set_xticklabels('', )

ax3.set_xlim([0, 170])
#ax3.set_ylim([0.4, 1.1])
ax3.set_ylim([-2.8, 1.9])

ax3.set_yticks([-2, 0, 2]) 
ax3.set_yticklabels(['-2', '0', '2'],  fontsize = tickFontSize) 
ax3.legend(ncol=4, loc=(+0.185, -1.75),frameon=False, fontsize=legendFontSize)

#+++++++++++++++++++++++++++++++++++++
#+++++++++++++++++++++++++++++++++++++

ax4 = axs[3]
ln42=ax4.plot(range(171), param_winterAnom_lowMissi, label="Winter", color = 'green', linestyle='-', linewidth=1.0)
ln43=ax4.plot(range(171), param_springAnom_lowMissi, label="Spring", color = 'blue', linestyle='-', linewidth=1.0)
ln44=ax4.plot(range(171), param_summerAnom_lowMissi, label="Summer", color = 'orange', linestyle='-', linewidth=1.0)
ln45=ax4.plot(range(171), param_fallAnom_lowMissi, label="Fall", color = 'red', linestyle='-', linewidth=1.0)
ln46=ax4.axhline(y = 0, color = 'k', label = '', linestyle='--', linewidth=0.7)
ln47=ax4.axvspan(1, 84.5, color='b',alpha=0.03)

ax4.set_title('(d) Lower Mississippi', y=0.98, pad=-14, fontsize = textFontSize, loc='left')

ax4.xaxis.set_major_formatter(plt.FixedFormatter(yMD))
ax4.set_xticks(timeTicks)
ax4.set_xticklabels(timeTickLebels, minor=False, rotation=0, fontsize = tickFontSize)

ax4.set_xlim([0, 170])
ax4.set_ylim([-1.9, 1.2])

ax4.set_yticks([-1, 0]) 
ax4.set_yticklabels(['-1', '0'],  fontsize = tickFontSize) 

ax4.set_xlabel('Time', fontsize = textFontSize)

#+++++++++++++++++++++++++++++++++++++
#+++++++++++++++++++++++++++++++++++++

plt.show()

#plt.savefig(r'C:\rezaul\1.academic\1.research\2.project\mrb\output\fig\paperOne\param\soilMoist\seasonal\soilMoistSeasonalAnomTs.pdf') 
#plt.savefig(r'C:\rezaul\1.academic\1.research\2.project\mrb\output\fig\paperOne\param\soilMoist\seasonal\soilMoistSeasonalAnomTs.png', dpi=600)  




In [68]:
STOP here ===========

SyntaxError: invalid syntax (<ipython-input-68-665768e7338e>, line 1)